# 4.- Training

In [41]:
import yaml
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml import command
from azure.ai.ml.entities import Environment, CommandJob
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input

In [42]:
# Load configuration from the YAML file
with open("../config.yaml", "r") as file:
    config = yaml.safe_load(file)

In [43]:
subscription_id = config["azure"]["subscription_id"]
resource_group = config["azure"]["resource_group_name"]
workspace_name = config["azure"]["workspace_name"]
container_name = config["azure"]["container_name"]
location = config["azure"]["location"]
datastore_name = config["azure"]["datastore_name"]
training_gpu_cluster = config["azure"]["training_gpu_cluster"]

In [44]:
# Asumiendo que tienes un datastore ya configurado y vinculado a tu espacio de trabajo
# datastore_name = 'your_datastore_name'
data_folder = f'{datastore_name}/data'
scripts_folder = f'{datastore_name}/scripts'

In [45]:
credential = DefaultAzureCredential()
ml_client = MLClient(credential, subscription_id, resource_group, workspace_name)

Overriding of current TracerProvider is not allowed
Overriding of current LoggerProvider is not allowed
Overriding of current MeterProvider is not allowed
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented
Attempting to instrument while already instrumented


In [53]:
# Define the local code folder (the training code will be automatically uploaded)
code_folder = "../src"

# Define the relative paths inside your code folder
# Adjust these paths according to your repository structure.
pipeline_config_path = f"{code_folder}/custom_pipeline_file.config"
# model_output_path = "models/research/object_detection/model_output"  # This folder will be used for model outputs
model_output_path = "../external/model_trained"  # This folder will be used for model outputs


# Set training parameters
num_train_steps = 1000

# Create the training job command
job = command(
    
    code=code_folder,  # The folder containing your code (it will be packaged and uploaded)
    command="""export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/research:`pwd`/research/slim && \
        pip install tf-slim && \
        pip install pycocotools && \
        python research/object_detection/model_main_tf2.py \
        --pipeline_config_path ${{inputs.pipeline_config_path}} \
        --model_dir ${{inputs.model_dir}} \
        --alsologtostderr \
        --num_train_steps ${{inputs.num_train_steps}} \
        --sample_1_of_n_eval_examples ${{inputs.sample_1_of_n_eval_examples}}""",
    inputs={
        "pipeline_config_path": Input(type="uri_file", path=pipeline_config_path),
        "model_dir": Input(type="uri_folder", path=model_output_path),
        "num_train_steps": num_train_steps,
        "sample_1_of_n_eval_examples": 1
    },
    environment="AzureML-tensorflow-2.12-cuda11@latest",  # Use the curated environment provided by Azure or your custom environment name
    compute="training-gpu-cluster",  # Replace with the name of your compute target
    experiment_name="tf-object-detection-training",
    display_name="TensorFlow Object Detection Training"
)


In [54]:
# Submit the job to Azure ML
job = ml_client.jobs.create_or_update(job)
print(f"Job submitted successfully. You can monitor it at: {job.url}")

Job submitted successfully. You can monitor it at: {}
